In [2]:
import sqlite3
import pandas as pd

In [2]:
df = pd.read_csv("Warehouse_and_Retail_Sales.csv")
print("Shape:", df.shape)  # (rows, columns)
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])

Shape: (307645, 9)
Rows: 307645
Columns: 9


In [3]:
df = pd.read_csv("Warehouse_and_Retail_Sales.csv")

# Display the first 5 rows
print(df.head())

   YEAR  MONTH                           SUPPLIER ITEM CODE  \
0  2020      1  REPUBLIC NATIONAL DISTRIBUTING CO    100009   
1  2020      1                          PWSWN INC    100024   
2  2020      1            RELIABLE CHURCHILL LLLP      1001   
3  2020      1          LANTERNA DISTRIBUTORS INC    100145   
4  2020      1               DIONYSOS IMPORTS INC    100293   

                      ITEM DESCRIPTION ITEM TYPE  RETAIL SALES  \
0                  BOOTLEG RED - 750ML      WINE          0.00   
1            MOMENT DE PLAISIR - 750ML      WINE          0.00   
2  S SMITH ORGANIC PEAR CIDER - 18.7OZ      BEER          0.00   
3        SCHLINK HAUS KABINETT - 750ML      WINE          0.00   
4       SANTORINI GAVALA WHITE - 750ML      WINE          0.82   

   RETAIL TRANSFERS  WAREHOUSE SALES  
0               0.0              2.0  
1               1.0              4.0  
2               0.0              1.0  
3               0.0              1.0  
4               0.0          

In [4]:
df = pd.read_csv("Warehouse_and_Retail_Sales.csv")

# Display a summary of missing values per column
missing_values = df.isnull().sum()
print(missing_values)

YEAR                  0
MONTH                 0
SUPPLIER            167
ITEM CODE             0
ITEM DESCRIPTION      0
ITEM TYPE             1
RETAIL SALES          3
RETAIL TRANSFERS      0
WAREHOUSE SALES       0
dtype: int64


In [5]:
# Drop rows with missing supplier, item type, or retail sales
df_cleaned = df.dropna(subset=['SUPPLIER', 'ITEM TYPE', 'RETAIL SALES']).reset_index(drop=True)

# Check new shape
print("After cleaning, shape:", df_cleaned.shape)

# Optional: Confirm no missing values remain
print("\nRemaining missing values:\n", df_cleaned.isnull().sum())

After cleaning, shape: (307477, 9)

Remaining missing values:
 YEAR                0
MONTH               0
SUPPLIER            0
ITEM CODE           0
ITEM DESCRIPTION    0
ITEM TYPE           0
RETAIL SALES        0
RETAIL TRANSFERS    0
WAREHOUSE SALES     0
dtype: int64


In [6]:
# Step 1: Load the CSV
df = pd.read_csv("Warehouse_and_Retail_Sales.csv")

# Step 2: Drop missing values (same as before)
df_cleaned = df.dropna(subset=['SUPPLIER', 'ITEM TYPE', 'RETAIL SALES']).reset_index(drop=True)

# Step 3: Connect to SQLite database (creates one if not exists)
conn = sqlite3.connect("warehouse_sales.db")

# Step 4: Write the cleaned dataframe into a raw table
df_cleaned.to_sql("RawSales", conn, if_exists="replace", index=False)

# Confirm
print("Raw table 'RawSales' written to SQLite DB.")

Raw table 'RawSales' written to SQLite DB.


In [2]:
db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query to list all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Print the tables
print("Tables in database:")
for table in tables:
    print(table[0])

Tables in database:
RawSales
Suppliers
sqlite_sequence
ItemTypes
Items


In [3]:
cursor.execute("PRAGMA table_info(RawSales);")
columns = cursor.fetchall()

# Print column names
print("Columns in RawSales:")
for col in columns:
    print(col[1])

Columns in RawSales:
YEAR
MONTH
SUPPLIER
ITEM CODE
ITEM DESCRIPTION
ITEM TYPE
RETAIL SALES
RETAIL TRANSFERS
WAREHOUSE SALES


In [4]:
db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# --- 1. Drop and Create Suppliers Table ---
cursor.execute("DROP TABLE IF EXISTS Suppliers;")
cursor.execute("""
CREATE TABLE Suppliers (
    supplier_id INTEGER PRIMARY KEY AUTOINCREMENT,
    supplier_name TEXT UNIQUE
);
""")
print("✅ Table 'Suppliers' created.")

# --- 2. Insert Unique Supplier Names from RawSales ---
cursor.execute("""
INSERT INTO Suppliers (supplier_name)
SELECT DISTINCT SUPPLIER
FROM RawSales
WHERE SUPPLIER IS NOT NULL;
""")
conn.commit()
print("Unique suppliers inserted.")

# --- 3. Preview the Table ---
suppliers_df = pd.read_sql_query("SELECT * FROM Suppliers LIMIT 5;", conn)
print("Preview of 'Suppliers' table:")
print(suppliers_df)

# --- 4. Close connection ---
conn.close()

✅ Table 'Suppliers' created.
Unique suppliers inserted.
Preview of 'Suppliers' table:
   supplier_id                      supplier_name
0            1  REPUBLIC NATIONAL DISTRIBUTING CO
1            2                          PWSWN INC
2            3            RELIABLE CHURCHILL LLLP
3            4          LANTERNA DISTRIBUTORS INC
4            5               DIONYSOS IMPORTS INC


In [5]:
db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# --- 1. Drop and Create ItemTypes Table ---
cursor.execute("DROP TABLE IF EXISTS ItemTypes;")
cursor.execute("""
CREATE TABLE ItemTypes (
    type_id INTEGER PRIMARY KEY AUTOINCREMENT,
    item_type TEXT UNIQUE
);
""")
print("Table 'ItemTypes' created.")

# --- 2. Insert Unique Item Types from RawSales ---
cursor.execute("""
INSERT INTO ItemTypes (item_type)
SELECT DISTINCT [ITEM TYPE]
FROM RawSales
WHERE [ITEM TYPE] IS NOT NULL;
""")
conn.commit()
print("Unique item types inserted.")

# --- 3. Preview the Table ---
itemtypes_df = pd.read_sql_query("SELECT * FROM ItemTypes LIMIT 5;", conn)
print("Preview of 'ItemTypes' table:")
print(itemtypes_df)

# --- 4. Close connection ---
conn.close()

Table 'ItemTypes' created.
Unique item types inserted.
Preview of 'ItemTypes' table:
   type_id     item_type
0        1          WINE
1        2          BEER
2        3        LIQUOR
3        4  STR_SUPPLIES
4        5          KEGS


In [14]:
# Check for item codes that map to more than one description or item type
conflicts_df = pd.read_sql_query("""
SELECT [ITEM CODE], COUNT(DISTINCT [ITEM DESCRIPTION]) AS desc_count, COUNT(DISTINCT [ITEM TYPE]) AS type_count
FROM RawSales
GROUP BY [ITEM CODE]
HAVING desc_count > 1 OR type_count > 1;
""", conn)

print("Conflicting ITEM CODEs:")
print(conflicts_df.head())
print("Total conflicts found:", len(conflicts_df))
conn.close()

Conflicting ITEM CODEs:
  ITEM CODE  desc_count  type_count
0      1001           2           1
1    101664           2           1
2     10356           2           1
3     10431           2           1
4     10444           2           1
Total conflicts found: 1021


In [3]:
db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# --- 1. Drop and Create Items Table with Composite PK ---
cursor.execute("DROP TABLE IF EXISTS Items;")
cursor.execute("""
CREATE TABLE Items (
    item_code TEXT,
    item_description TEXT,
    type_id INTEGER,
    PRIMARY KEY (item_code, item_description),
    FOREIGN KEY(type_id) REFERENCES ItemTypes(type_id)
);
""")
print("Table 'Items' created.")

# --- 2. Insert Unique Items without violating PK constraints ---
cursor.execute("""
INSERT INTO Items (item_code, item_description, type_id)
SELECT item_code, item_description, type_id
FROM (
    SELECT 
        rs.[ITEM CODE] AS item_code,
        rs.[ITEM DESCRIPTION] AS item_description,
        it.type_id,
        ROW_NUMBER() OVER (
            PARTITION BY rs.[ITEM CODE], rs.[ITEM DESCRIPTION]
            ORDER BY COUNT(*) DESC
        ) as rn
    FROM RawSales rs
    JOIN ItemTypes it ON rs.[ITEM TYPE] = it.item_type
    GROUP BY rs.[ITEM CODE], rs.[ITEM DESCRIPTION], it.type_id
)
WHERE rn = 1;
""")
conn.commit()
print("Unique items inserted.")

# --- 3. Preview the Table ---
items_df = pd.read_sql_query("SELECT * FROM Items LIMIT 5;", conn)
print("Preview of 'Items' table:")
print(items_df)

# --- 4. Close connection ---
conn.close()

Table 'Items' created.
Unique items inserted.
Preview of 'Items' table:
  item_code                     item_description  type_id
0    100002       PATRON TEQUILA SILVER LTD - 1L        3
1    100007            LA CETTO CAB SAUV - 750ML        1
2    100008  AMITY VINEYARDS P/NOIR 2013 - 750ML        1
3    100009                  BOOTLEG RED - 750ML        1
4    100011                   PAPI P/GRIG - 1.5L        1


In [4]:
# Reconnect
db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# --- 1. Drop and Create Sales Table ---
cursor.execute("DROP TABLE IF EXISTS Sales;")
cursor.execute("""
CREATE TABLE Sales (
    sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
    year INTEGER,
    month INTEGER,
    item_code TEXT,
    item_description TEXT,
    supplier_id INTEGER,
    retail_sales REAL,
    retail_transfers REAL,
    warehouse_sales REAL,
    FOREIGN KEY(item_code, item_description) REFERENCES Items(item_code, item_description),
    FOREIGN KEY(supplier_id) REFERENCES Suppliers(supplier_id)
);
""")
print("Table 'Sales' created.")

# --- 2. Insert Data from RawSales with JOINs to foreign keys ---
cursor.execute("""
INSERT INTO Sales (
    year, month, item_code, item_description, supplier_id,
    retail_sales, retail_transfers, warehouse_sales
)
SELECT
    rs.YEAR,
    rs.MONTH,
    rs.[ITEM CODE],
    rs.[ITEM DESCRIPTION],
    s.supplier_id,
    rs.[RETAIL SALES],
    rs.[RETAIL TRANSFERS],
    rs.[WAREHOUSE SALES]
FROM RawSales rs
JOIN Suppliers s ON rs.SUPPLIER = s.supplier_name
JOIN Items i ON rs.[ITEM CODE] = i.item_code AND rs.[ITEM DESCRIPTION] = i.item_description;
""")
conn.commit()
print("Sales data inserted.")

# --- 3. Preview the Table ---
sales_df = pd.read_sql_query("SELECT * FROM Sales LIMIT 5;", conn)
print("Preview of 'Sales' table:")
print(sales_df)

# --- 4. Close connection ---
conn.close()

Table 'Sales' created.
Sales data inserted.
Preview of 'Sales' table:
   sale_id  year  month item_code                     item_description  \
0        1  2020      1    100009                  BOOTLEG RED - 750ML   
1        2  2020      1    100024            MOMENT DE PLAISIR - 750ML   
2        3  2020      1      1001  S SMITH ORGANIC PEAR CIDER - 18.7OZ   
3        4  2020      1    100145        SCHLINK HAUS KABINETT - 750ML   
4        5  2020      1    100293       SANTORINI GAVALA WHITE - 750ML   

   supplier_id  retail_sales  retail_transfers  warehouse_sales  
0            1          0.00               0.0              2.0  
1            2          0.00               1.0              4.0  
2            3          0.00               0.0              1.0  
3            4          0.00               0.0              1.0  
4            5          0.82               0.0              0.0  


In [5]:
db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Step 1: List all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]

# Step 2: Print dimensions and column names
print("TABLE SUMMARY\n" + "-"*50)
for table in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table} LIMIT 5;", conn)
    total_rows = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table};", conn)['count'][0]
    print(f"Table: {table}")
    print(f" - Rows: {total_rows}")
    print(f" - Columns ({len(df.columns)}): {list(df.columns)}\n")

# Close connection
conn.close()

TABLE SUMMARY
--------------------------------------------------
Table: RawSales
 - Rows: 307477
 - Columns (9): ['YEAR', 'MONTH', 'SUPPLIER', 'ITEM CODE', 'ITEM DESCRIPTION', 'ITEM TYPE', 'RETAIL SALES', 'RETAIL TRANSFERS', 'WAREHOUSE SALES']

Table: sqlite_sequence
 - Rows: 3
 - Columns (2): ['name', 'seq']

Table: Suppliers
 - Rows: 396
 - Columns (2): ['supplier_id', 'supplier_name']

Table: ItemTypes
 - Rows: 8
 - Columns (2): ['type_id', 'item_type']

Table: Items
 - Rows: 35122
 - Columns (3): ['item_code', 'item_description', 'type_id']

Table: Sales
 - Rows: 307477
 - Columns (9): ['sale_id', 'year', 'month', 'item_code', 'item_description', 'supplier_id', 'retail_sales', 'retail_transfers', 'warehouse_sales']



In [6]:

# Reconnect to the database
db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Check earliest and latest year/month
cursor.execute("""
SELECT 
    MIN(year) AS earliest_year,
    MIN(month) AS earliest_month,
    MAX(year) AS latest_year,
    MAX(month) AS latest_month
FROM Sales;
""")

date_range = cursor.fetchone()
conn.close()

print(f"Earliest date: {date_range[0]}-{str(date_range[1]).zfill(2)}")
print(f"Latest date:   {date_range[2]}-{str(date_range[3]).zfill(2)}")

conn.close()

Earliest date: 2017-01
Latest date:   2020-12


In [18]:
# Question 1: Which item types generated the highest total retail sales over time?

# Connect to the database
db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Execute SQL query using JOINs to get total retail sales by item type
cursor.execute("""
SELECT 
    it.item_type,
    ROUND(SUM(s.retail_sales), 2) AS total_retail_sales
FROM Sales s
JOIN Items i 
    ON s.item_code = i.item_code AND s.item_description = i.item_description
JOIN ItemTypes it 
    ON i.type_id = it.type_id
GROUP BY it.item_type
ORDER BY total_retail_sales DESC;
""")

# Fetch and convert to DataFrame
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
q1_result = pd.DataFrame(rows, columns=columns)

# Display result
print("Question 1: Total Retail Sales by Item Type (Over Time)")
print(q1_result)

# Close connection
conn.close()

Question 1: Total Retail Sales by Item Type (Over Time)
      item_type  total_retail_sales
0        LIQUOR           802730.17
1          WINE           746402.40
2          BEER           574274.46
3   NON-ALCOHOL            27153.83
4  STR_SUPPLIES             2235.54
5           REF              662.99
6          KEGS                0.00
7       DUNNAGE                0.00


In [19]:
# Pandas version of Question 1: Total retail sales by item type (2017–2020)

import sqlite3
import pandas as pd

# Reconnect to your database (if not already connected)
db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)

# Load the necessary tables into DataFrames
sales_df = pd.read_sql_query("SELECT * FROM Sales;", conn)
items_df = pd.read_sql_query("SELECT * FROM Items;", conn)
itemtypes_df = pd.read_sql_query("SELECT * FROM ItemTypes;", conn)

# Merge: Sales → Items → ItemTypes
merged_df = (
    sales_df
    .merge(items_df, on=['item_code', 'item_description'], how='left')
    .merge(itemtypes_df, on='type_id', how='left')
)

# Group and aggregate
q1_pandas = (
    merged_df
    .groupby('item_type')['retail_sales']
    .sum()
    .round(2)
    .sort_values(ascending=False)
    .reset_index()
)

# Display result
print("Pandas result for Question 1: Total Retail Sales by Item Type Over Time")
print(q1_pandas)

# Close connection
conn.close()

Pandas result for Question 1: Total Retail Sales by Item Type Over Time
      item_type  retail_sales
0        LIQUOR     802730.17
1          WINE     746402.40
2          BEER     574274.46
3   NON-ALCOHOL      27153.83
4  STR_SUPPLIES       2235.54
5           REF        662.99
6       DUNNAGE          0.00
7          KEGS          0.00


In [17]:
# Question 2: Which suppliers generated the highest total retail sales over time?

# Connect to the database
db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
SELECT 
    s.supplier_name,
    ROUND(SUM(sa.retail_sales), 2) AS total_retail_sales
FROM Sales sa
JOIN Suppliers s ON sa.supplier_id = s.supplier_id
GROUP BY s.supplier_name
ORDER BY total_retail_sales DESC
LIMIT 10;
""")

# Fetch and convert to DataFrame
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
q2_result = pd.DataFrame(rows, columns=columns)

# Display result
print("Question 2: Top 10 Suppliers by Retail Sales (All Time)")
print(q2_result)

# Close connection
conn.close()

Question 2: Top 10 Suppliers by Retail Sales (All Time)
              supplier_name  total_retail_sales
0        E & J GALLO WINERY           166170.53
1  DIAGEO NORTH AMERICA INC           145343.20
2      CONSTELLATION BRANDS           131664.79
3        ANHEUSER BUSCH INC           109960.82
4        JIM BEAM BRANDS CO            96164.04
5    MILLER BREWING COMPANY            87156.44
6             CROWN IMPORTS            84437.66
7                SAZERAC CO            70414.62
8           BACARDI USA INC            66610.23
9            THE WINE GROUP            64270.04


In [10]:
# Pandas version of Question 2: Which suppliers generated the highest total retail sales over time?


db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)

# Load the necessary tables
sales_df = pd.read_sql_query("SELECT * FROM Sales;", conn)
suppliers_df = pd.read_sql_query("SELECT * FROM Suppliers;", conn)

# Merge Sales with Suppliers
merged_df = sales_df.merge(suppliers_df, on="supplier_id", how="left")

# Group and aggregate
q2_pandas = (
    merged_df
    .groupby('supplier_name')['retail_sales']
    .sum()
    .round(2)
    .sort_values(ascending=False)
    .reset_index()
    .head(10)
)

# Display result
print("Pandas result for Question 2: Top 10 Suppliers by Retail Sales (All Time)")
print(q2_pandas)

# Close connection
conn.close()

Pandas result for Question 2: Top 10 Suppliers by Retail Sales (All Time)
              supplier_name  retail_sales
0        E & J GALLO WINERY     166170.53
1  DIAGEO NORTH AMERICA INC     145343.20
2      CONSTELLATION BRANDS     131664.79
3        ANHEUSER BUSCH INC     109960.82
4        JIM BEAM BRANDS CO      96164.04
5    MILLER BREWING COMPANY      87156.44
6             CROWN IMPORTS      84437.66
7                SAZERAC CO      70414.62
8           BACARDI USA INC      66610.23
9            THE WINE GROUP      64270.04


In [11]:
# Question 3: Which months show the largest gap between retail sales and warehouse sales?


db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
SELECT 
    year,
    month,
    ROUND(SUM(retail_sales), 2) AS total_retail_sales,
    ROUND(SUM(warehouse_sales), 2) AS total_warehouse_sales,
    ROUND(SUM(retail_sales - warehouse_sales), 2) AS sales_gap
FROM Sales
GROUP BY year, month
ORDER BY ABS(sales_gap) DESC
LIMIT 10;
""")

# Fetch results and convert to DataFrame
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
q3_result = pd.DataFrame(rows, columns=columns)

# Display result
print("Question 3: Top 10 Months with the Largest Retail vs Warehouse Sales Gap")
print(q3_result)

# Close the connection
conn.close()

Question 3: Top 10 Months with the Largest Retail vs Warehouse Sales Gap
   year  month  total_retail_sales  total_warehouse_sales  sales_gap
0  2020      7            91766.60              418320.42 -326553.82
1  2017      8            87111.78              382895.69 -295783.91
2  2019      5            94921.74              383857.58 -288935.84
3  2020      9            80643.26              365417.61 -284774.35
4  2019      7            90730.72              375427.48 -284696.76
5  2017      6            97357.26              379465.83 -282108.57
6  2019      8            90603.58              357517.09 -266913.51
7  2019     10            87316.63              350638.03 -263321.40
8  2019      6            90821.64              346405.08 -255583.44
9  2017     11            98316.19              341440.66 -243124.47


In [12]:
# Pandas version of Question 3: Which months show the largest gap between retail sales and warehouse sales?


db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)

# Load the Sales table into a DataFrame
sales_df = pd.read_sql_query("SELECT * FROM Sales;", conn)

# Group by year and month and calculate sums
q3_pandas = (
    sales_df
    .groupby(['year', 'month'])[['retail_sales', 'warehouse_sales']]
    .sum()
    .round(2)
    .reset_index()
)

# Calculate sales gap
q3_pandas['sales_gap'] = q3_pandas['retail_sales'] - q3_pandas['warehouse_sales']

# Sort by absolute gap
q3_pandas = q3_pandas.reindex(q3_pandas['sales_gap'].abs().sort_values(ascending=False).index)

# Take top 10
q3_pandas_top10 = q3_pandas.head(10)

# Display result
print("Pandas result for Question 3: Top 10 Months with the Largest Retail vs Warehouse Sales Gap")
print(q3_pandas_top10)

# Close connection
conn.close()


Pandas result for Question 3: Top 10 Months with the Largest Retail vs Warehouse Sales Gap
    year  month  retail_sales  warehouse_sales  sales_gap
22  2020      7      91766.60        418320.42 -326553.82
2   2017      8      87111.78        382895.69 -295783.91
13  2019      5      94921.74        383857.58 -288935.84
23  2020      9      80643.26        365417.61 -284774.35
15  2019      7      90730.72        375427.48 -284696.76
0   2017      6      97357.26        379465.83 -282108.57
16  2019      8      90603.58        357517.09 -266913.51
18  2019     10      87316.63        350638.03 -263321.40
14  2019      6      90821.64        346405.08 -255583.44
5   2017     11      98316.19        341440.66 -243124.47


In [13]:
# Question 4: Which items had high warehouse sales but very low retail sales?


db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Execute the SQL query
cursor.execute("""
SELECT 
    s.item_code,
    s.item_description,
    ROUND(SUM(s.retail_sales), 2) AS total_retail_sales,
    ROUND(SUM(s.warehouse_sales), 2) AS total_warehouse_sales,
    ROUND(SUM(s.warehouse_sales - s.retail_sales), 2) AS unused_inventory
FROM Sales s
GROUP BY s.item_code, s.item_description
HAVING total_retail_sales < 10 AND total_warehouse_sales > 500
ORDER BY unused_inventory DESC
LIMIT 10;
""")

# Fetch results and convert to DataFrame
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
q4_result = pd.DataFrame(rows, columns=columns)

# Display result
print("Question 4: Top 10 Underperforming Items with High Warehouse Volume")
print(q4_result)

# Close connection
conn.close()


Question 4: Top 10 Underperforming Items with High Warehouse Volume
  item_code                    item_description  total_retail_sales  \
0     91545  MILWAUKEE BEST ICE 2/12 CAN - 12OZ                0.00   
1     90921        MILLER LITE 18PK LNNR - 12OZ                0.00   
2     31126            STEEL RESERVE 12/42OZ NR                1.42   
3     92223    STEEL RESERVE 12/24OZ CAN - 24OZ                0.00   
4     20090       MODELO ESPECIAL 18PK 12.0Z NR                0.00   
5     26145            BUD 18PK 16OZ CAN - 16OZ                0.00   
6      7242                   BUD LIGHT 18PK NR                8.00   
7     94242     KEYSTONE ICE 12/24OZ CAN - 24OZ                0.00   
8     26305             BUD 15/25OZ CANS - 25OZ                0.00   
9     30823             NATTY DADDY 15/25OZ CAN                0.00   

   total_warehouse_sales  unused_inventory  
0               31200.36          31200.36  
1               23964.82          23964.82  
2              

In [14]:
# Pandas version of Question 4: Which items had high warehouse sales but very low retail sales?


db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)

# Load the Sales table
sales_df = pd.read_sql_query("SELECT * FROM Sales;", conn)

# Group by item_code and item_description
item_sales = (
    sales_df
    .groupby(['item_code', 'item_description'])[['retail_sales', 'warehouse_sales']]
    .sum()
    .round(2)
    .reset_index()
)

# Calculate the sales gap
item_sales['unused_inventory'] = item_sales['warehouse_sales'] - item_sales['retail_sales']

# Filter for underperforming items
q4_pandas = (
    item_sales[
        (item_sales['retail_sales'] < 10) &
        (item_sales['warehouse_sales'] > 500)
    ]
    .sort_values(by='unused_inventory', ascending=False)
    .head(10)
)

# Display result
print("Pandas result for Question 4: Top 10 Underperforming Items with High Warehouse Volume")
print(q4_pandas)

# Close the connection
conn.close()


Pandas result for Question 4: Top 10 Underperforming Items with High Warehouse Volume
      item_code                    item_description  retail_sales  \
34604     91545  MILWAUKEE BEST ICE 2/12 CAN - 12OZ          0.00   
34575     90921        MILLER LITE 18PK LNNR - 12OZ          0.00   
10935     31126            STEEL RESERVE 12/42OZ NR          1.42   
34642     92223    STEEL RESERVE 12/24OZ CAN - 24OZ          0.00   
3167      20090       MODELO ESPECIAL 18PK 12.0Z NR          0.00   
7851      26145            BUD 18PK 16OZ CAN - 16OZ          0.00   
31449      7242                   BUD LIGHT 18PK NR          8.00   
34749     94242     KEYSTONE ICE 12/24OZ CAN - 24OZ          0.00   
7919      26305             BUD 15/25OZ CANS - 25OZ          0.00   
10147     30823             NATTY DADDY 15/25OZ CAN          0.00   

       warehouse_sales  unused_inventory  
34604         31200.36          31200.36  
34575         23964.82          23964.82  
10935         21811.01   

In [15]:
# Question 5: Which item type generated the highest total retail sales in each year?

# Connect to the database
db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Execute the SQL query with joins and rank logic
cursor.execute("""
WITH yearly_sales_by_type AS (
    SELECT 
        s.year,
        it.item_type,
        ROUND(SUM(s.retail_sales), 2) AS total_retail_sales
    FROM Sales s
    JOIN Items i ON s.item_code = i.item_code AND s.item_description = i.item_description
    JOIN ItemTypes it ON i.type_id = it.type_id
    GROUP BY s.year, it.item_type
)

SELECT *
FROM (
    SELECT *,
           RANK() OVER (PARTITION BY year ORDER BY total_retail_sales DESC) AS rank
    FROM yearly_sales_by_type
)
WHERE rank = 1;
""")

# Fetch and load to DataFrame
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
q5_result = pd.DataFrame(rows, columns=columns)

# Display result
print("Question 5: Most Profitable Item Type per Year (Highest Retail Sales)")
print(q5_result)

# Close connection
conn.close()


Question 5: Most Profitable Item Type per Year (Highest Retail Sales)
   year item_type  total_retail_sales  rank
0  2017    LIQUOR           250689.19     1
1  2018      WINE            59190.40     1
2  2019    LIQUOR           357074.03     1
3  2020    LIQUOR           137523.06     1


In [16]:
# Pandas version of Question 5: Which item type generated the highest total retail sales in each year?

db_path = '/Users/khanzakirul/Documents/EAS 503/MP2_new/warehouse_sales.db'
conn = sqlite3.connect(db_path)

# Load required tables
sales_df = pd.read_sql_query("SELECT * FROM Sales;", conn)
items_df = pd.read_sql_query("SELECT * FROM Items;", conn)
itemtypes_df = pd.read_sql_query("SELECT * FROM ItemTypes;", conn)

# Merge: Sales → Items → ItemTypes
merged_df = (
    sales_df
    .merge(items_df, on=['item_code', 'item_description'], how='left')
    .merge(itemtypes_df, on='type_id', how='left')
)

# Group by year and item_type, aggregate retail sales
grouped = (
    merged_df
    .groupby(['year', 'item_type'])['retail_sales']
    .sum()
    .reset_index()
    .round(2)
)

# Rank by year
grouped['rank'] = grouped.groupby('year')['retail_sales'].rank(method='dense', ascending=False)

# Filter top-ranked item types per year
q5_pandas = grouped[grouped['rank'] == 1].sort_values('year')

# Display result
print("Pandas result for Question 5: Most Profitable Item Type per Year (Highest Retail Sales)")
print(q5_pandas)

# Close connection
conn.close()


Pandas result for Question 5: Most Profitable Item Type per Year (Highest Retail Sales)
    year item_type  retail_sales  rank
3   2017    LIQUOR     250689.19   1.0
15  2018      WINE      59190.40   1.0
19  2019    LIQUOR     357074.03   1.0
27  2020    LIQUOR     137523.06   1.0
